In [12]:
from openff.evaluator.properties import Density

In [13]:
schema = Density.default_simulation_schema()

In [14]:
Density.default_simulation_schema().workflow_schema.protocol_schemas[0]

In [15]:
schema.get_attributes()

['absolute_tolerance', 'relative_tolerance', 'workflow_schema']

In [16]:
schema.workflow_schema.get_attributes()

['protocol_schemas',
 'protocol_replicators',
 'final_value_source',
 'outputs_to_store']

In [17]:
schema.workflow_schema.protocol_schemas[0].get_attributes()

['id', 'type', 'inputs']

In [18]:
for s in schema.workflow_schema.protocol_schemas:
    print(s.id,"\n",s.type,"\n",s.inputs,"\n")

build_coordinates 
 BuildCoordinatesPackmol 
 {'.allow_merging': True, '.max_molecules': 1000, '.count_exact_amount': True, '.mass_density': <Quantity(0.95, 'gram / milliliter')>, '.box_aspect_ratio': [1.0, 1.0, 1.0], '.substance': <ProtocolPath full_path=global.substance>, '.tolerance': <Quantity(2.0, 'angstrom')>, '.verbose_packmol': False, '.retain_packmol_files': False} 

assign_parameters 
 BaseBuildSystem 
 {'.allow_merging': True, '.force_field_path': <ProtocolPath full_path=global.force_field_path>, '.coordinate_file_path': <ProtocolPath full_path=build_coordinates.coordinate_file_path>, '.substance': <ProtocolPath full_path=build_coordinates.output_substance>} 

energy_minimisation 
 OpenMMEnergyMinimisation 
 {'.allow_merging': True, '.input_coordinate_file': <ProtocolPath full_path=build_coordinates.coordinate_file_path>, '.parameterized_system': <ProtocolPath full_path=assign_parameters.parameterized_system>, '.tolerance': <Quantity(10.0, 'kilojoules_per_mole / nanometer')>

In [19]:
schema.workflow_schema.protocol_schemas[6].inputs

{'.allow_merging': True,
 '.time_series_statistics': <ProtocolPath full_path=conditional_group/average_density.time_series_statistics>,
 '.input_observables': <ProtocolPath full_path=conditional_group/production_simulation.observables>}

In [20]:
schema.workflow_schema.final_value_source.protocol_ids

('conditional_group', 'average_density')

In [21]:
schema.workflow_schema.outputs_to_store

{'full_system': <openff.evaluator.storage.data.StoredSimulationData at 0x767e226e91d0>}

In [22]:
from openff.evaluator import unit
from openff.evaluator.workflow import Workflow, ProtocolPath
from openff.evaluator.workflow.schemas import WorkflowSchema
from openff.evaluator.protocols.simulation import ProductionSimulation
from openff.evaluator.protocols.analysis import AnalyzeTrajectory
from openff.evaluator.properties.density import Density

class CustomDensityWorkflow(Density):
    @staticmethod
    def default_simulation_schema(replicas: int = 1) -> WorkflowSchema:
        # Get the default Density workflow schema
        default_schema = Density.default_simulation_schema(replicas)

        # Initialize a new schema
        schema = WorkflowSchema()

        # Step 1: Copy build_coordinates from the default schema
        build_coordinates = default_schema.workflow_schema.protocol_schemas[0]
        schema.protocol_schemas.append(build_coordinates)

        # Step 2: Copy assign_parameters from the default schema
        assign_parameters = default_schema.workflow_schema.protocol_schemas[1]
        schema.protocol_schemas.append(assign_parameters)

        # Step 3: Copy energy_minimisation from the default schema
        energy_minimization = default_schema.workflow_schema.protocol_schemas[2]
        schema.protocol_schemas.append(energy_minimization)

        # Step 4: Copy equilibration_simulation from the default schema
        equilibration = default_schema.workflow_schema.protocol_schemas[3]
        schema.protocol_schemas.append(equilibration)

        # Step 5: Custom production simulation with frequent density checks
        production = ProductionSimulation("production_simulation")
        production.input_coordinate_file = ProtocolPath(
            "output_coordinate_file", equilibration.id
        )
        production.input_force_field = ProtocolPath(
            "parameterized_system", assign_parameters.id
        )
        production.steps = 1000000  # Adjust as needed for production
        production.check_interval = 10000  # Check density every 10,000 steps
        schema.protocol_schemas.append(production)

        # Step 6: Frequent density checks
        for i in range(0, production.steps, production.check_interval):
            density_check = Density("density_check_{}".format(i))
            density_check.input_coordinate_file = ProtocolPath(
                "output_coordinate_file", equilibration.id
            )
            density_check.input_force_field = ProtocolPath(
                "parameterized_system", assign_parameters.id
            )
            density_check.steps = production.check_interval
            density_check.output_to_store = ProtocolPath(
                "output_density", production.id
            )
            schema.protocol_schemas.append(density_check)

        # Step 7: Final production run if density is converged
        final_production = ProductionSimulation("final_production_simulation")
        final_production.input_coordinate_file = ProtocolPath(
            "output_coordinate_file", equilibration.id
        )
        final_production.input_force_field = ProtocolPath(
            "parameterized_system", assign_parameters.id
        )
        final_production.steps = 2500000  # 5 ns run assuming 2 fs timestep
        final_production.conditional_group = "density_converged"
        schema.protocol_schemas.append(final_production)

        # Step 8: Analysis step to check if density has converged
        density_analysis = AnalyzeTrajectory("density_analysis")
        density_analysis.input_coordinate_file = ProtocolPath(
            "output_coordinate_file", production.id
        )
        density_analysis.input_trajectory_path = ProtocolPath(
            "output_trajectory_path", production.id
        )
        density_analysis.output_to_store = ProtocolPath(
            "output_density", production.id
        )
        density_analysis.conditional_group = "density_converged"
        schema.protocol_schemas.append(density_analysis)

        return schema

# Example usage
if __name__ == "__main__":
    schema = CustomDensityWorkflow.default_simulation_schema()
    workflow = Workflow()
    workflow.schema = schema

    # Assign global variables and execute workflow as needed.


ImportError: cannot import name 'ProtocolPath' from 'openff.evaluator.workflow' (/localhome/cschiebroek/MDFP_VP/mdfptools/carl/openff-evaluator/openff/evaluator/workflow/__init__.py)

In [ ]:
default_schema = Density.default_simulation_schema()

# Initialize a new schema
schema = WorkflowSchema()
build_coordinates = default_schema.workflow_schema.protocol_schemas[0]


In [ ]:
default_schema.workflow_schema.protocol_schemas[0].get_attributes()

['id', 'type', 'inputs']

In [ ]:
import os
from rdkit import Chem
from rdkit.Chem import rdDistGeom
from openff.evaluator import unit
from openff.evaluator.client import EvaluatorClient, RequestOptions
from openff.evaluator.forcefield import ForceFieldSource
from openff.evaluator.workflow import Workflow
from openff.evaluator.workflow.utils import ProtocolPath
from openff.evaluator.workflow.schemas import WorkflowSchema
from openff.evaluator.protocols.utils import generate_simulation_protocols
from openff.evaluator.properties.density import Density
from openff.evaluator.substances import Substance, Component, MoleFraction
from openff.evaluator.thermodynamics import ThermodynamicState
from openff.toolkit.typing.engines.smirnoff import ForceField

from openff.evaluator.workflow.schemas import WorkflowSchema, ProtocolPath
from openff.evaluator.protocols.analysis import AverageObservable
from openff.evaluator.utils.observables import ObservableType, ObservableArray
from openff.units import unit as openff_unit

# Define the custom density workflow
class CustomDensityWorkflow(Density):
    @staticmethod
    def default_simulation_schema(replicas: int = 3) -> WorkflowSchema:
        # Create the analysis protocol

        # Create the analysis protocol
        analysis_protocol = AverageObservable("average_density")
        analysis_protocol.observable = ObservableArray([ObservableType.Density])

        
        # Generate the default simulation protocols
        protocols, _, _ = generate_simulation_protocols(
            analysis_protocol,
            use_target_uncertainty=True,
            id_suffix=""
        )

        schema = WorkflowSchema()
        schema.protocol_schemas.extend(protocols)

        # Custom production simulation with frequent density checks
        production_protocol = protocols.production_simulation
        production_protocol.steps_per_iteration = 10000  # Adjusted for shorter simulation time
        production_protocol.output_frequency = 1000  # Check density every 1,000 steps

        # Add frequent density checks
        for i in range(0, production_protocol.steps_per_iteration, production_protocol.output_frequency):
            density_check = Density("density_check_{}".format(i))
            density_check.input_coordinate_file = ProtocolPath(
                "output_coordinate_file", protocols.equilibration_simulation.id
            )
            density_check.input_force_field = ProtocolPath(
                "parameterized_system", protocols.assign_parameters.id
            )
            density_check.steps = production_protocol.output_frequency
            density_check.output_to_store = ProtocolPath(
                "output_density", production_protocol.id
            )
            schema.protocol_schemas.append(density_check)

        # Add a final production run if density is converged
        final_production = generate_simulation_protocols(
            analysis_protocol,
            use_target_uncertainty=False,
            id_suffix="final"
        ).production_simulation
        final_production.steps_per_iteration = 50000  # 0.1 ns run for testing
        final_production.conditional_group = "density_converged"
        schema.protocol_schemas.append(final_production)

        # Analysis step to check if density has converged
        density_analysis = Density("density_analysis")
        density_analysis.input_coordinate_file = ProtocolPath(
            "output_coordinate_file", production_protocol.id
        )
        density_analysis.input_trajectory_path = ProtocolPath(
            "output_trajectory_path", production_protocol.id
        )
        density_analysis.output_to_store = ProtocolPath(
            "output_density", production_protocol.id
        )
        density_analysis.conditional_group = "density_converged"
        schema.protocol_schemas.append(density_analysis)

        return schema


# Define the substance from rdkit
benzene_smiles = "c1ccccc1"
benzene_mol = Chem.AddHs(Chem.MolFromSmiles(benzene_smiles))
rdDistGeom.EmbedMolecule(benzene_mol)
smiles = Chem.MolToSmiles(benzene_mol)
benzene = Substance()
benzene.add_component(Component(smiles=smiles), MoleFraction(1.0))

# Define the thermodynamic state
thermodynamic_state = ThermodynamicState(
    temperature=298.15 * unit.kelvin,
    pressure=1.0 * unit.atmosphere
)

# Define the force field
forcefield = ForceField("openff_unconstrained-2.1.0.offxml")
forcefield_path = "openff_unconstrained-2.1.0.offxml"

# Define the workflow
schema = CustomDensityWorkflow.default_simulation_schema()
workflow = Workflow()
workflow.schema = schema

# Define evaluator client
client = EvaluatorClient()

# Define request options
options = RequestOptions()
options.calculation_layers = [workflow]

# Add global variables
workflow.schema.global_protocols["force_field_path"] = forcefield_path
workflow.schema.global_protocols["coordinate_file_path"] = "build_coordinates"

# Create and submit the estimation request
request = client.request_estimate(
    properties=[CustomDensityWorkflow()],
    substances=[benzene],
    thermodynamic_states=[thermodynamic_state],
    options=options,
)

# Wait for results
results = request.results()
print(results)


TypeError: The value must be a unit-wrapped integer, float or numpy array.

In [ ]:
import os
from rdkit import Chem
from rdkit.Chem import rdDistGeom
from openff.evaluator import unit
from openff.evaluator.client import EvaluatorClient, RequestOptions
from openff.evaluator.forcefield import ForceFieldSource
from openff.evaluator.workflow import Workflow
from openff.evaluator.workflow.utils import ProtocolPath

from openff.evaluator.workflow.schemas import WorkflowSchema
from openff.evaluator.properties.density import Density
from openff.evaluator.protocols.coordinates import BuildCoordinatesPackmol
from openff.evaluator.protocols.forcefield import BuildSmirnoffSystem
from openff.evaluator.protocols.openmm import OpenMMSimulation, OpenMMEnergyMinimisation

from openff.evaluator.utils.observables import ObservableType, ObservableArray
from openff.evaluator.substances import Substance, Component, MoleFraction
from openff.evaluator.thermodynamics import ThermodynamicState
from openff.toolkit.typing.engines.smirnoff import ForceField

# Define the custom density workflow
class CustomDensityWorkflow(Density):
    @staticmethod
    def default_simulation_schema(replicas: int = 3) -> WorkflowSchema:
        schema = WorkflowSchema()

        # Step 1: Build coordinates
        build_coordinates = BuildCoordinatesPackmol("build_coordinates")
        build_coordinates.max_molecules = 1000
        build_coordinates.mass_density = 0.95 * unit.grams / unit.milliliters
        schema.protocol_schemas.append(build_coordinates)

        # Step 2: Assign parameters
        assign_parameters = BuildSmirnoffSystem("assign_parameters")
        assign_parameters.force_field_path = ProtocolPath("force_field_path", "global")
        assign_parameters.coordinate_file_path = ProtocolPath("coordinate_file_path", build_coordinates.id)
        schema.protocol_schemas.append(assign_parameters)

        # Step 3: Energy minimization
        energy_minimization = OpenMMEnergyMinimisation("energy_minimisation")
        energy_minimization.input_coordinate_file = ProtocolPath("coordinate_file_path", assign_parameters.id)
        energy_minimization.input_force_field = ProtocolPath("parameterized_system", assign_parameters.id)
        schema.protocol_schemas.append(energy_minimization)

        # Step 4: Equilibration simulation
        equilibration_simulation = OpenMMSimulation("equilibration_simulation")
        equilibration_simulation.input_coordinate_file = ProtocolPath("output_coordinate_file", energy_minimization.id)
        equilibration_simulation.input_force_field = ProtocolPath("parameterized_system", assign_parameters.id)
        equilibration_simulation.steps = 100000
        schema.protocol_schemas.append(equilibration_simulation)

        # Step 5: Production simulation with frequent checks
        production_simulation = OpenMMSimulation("production_simulation")
        production_simulation.input_coordinate_file = ProtocolPath("output_coordinate_file", equilibration_simulation.id)
        production_simulation.input_force_field = ProtocolPath("parameterized_system", assign_parameters.id)
        production_simulation.steps = 10000  # Shorter simulation time for testing
        schema.protocol_schemas.append(production_simulation)

        # Step 6: Frequent density checks
        for i in range(0, production_simulation.steps, 1000):
            density_check = AverageObservable(f"density_check_{i}")
            density_check.observable = ObservableType.Density
            density_check.thermodynamic_state = ProtocolPath("thermodynamic_state", "global")
            density_check.input_coordinate_file = ProtocolPath("output_coordinate_file", equilibration_simulation.id)
            density_check.input_trajectory_path = ProtocolPath("trajectory_file_path", production_simulation.id)
            schema.protocol_schemas.append(density_check)

        # Step 7: Final production run after density convergence
        final_production = OpenMMSimulation("final_production_simulation")
        final_production.input_coordinate_file = ProtocolPath("output_coordinate_file", equilibration_simulation.id)
        final_production.input_force_field = ProtocolPath("parameterized_system", assign_parameters.id)
        final_production.steps = 50000  # 0.1 ns run for testing
        final_production.conditional_group = "density_converged"
        schema.protocol_schemas.append(final_production)

        # Analysis step to check if density has converged
        density_analysis = AverageObservable("density_analysis")
        density_analysis.observable = ObservableType.Density
        density_analysis.thermodynamic_state = ProtocolPath("thermodynamic_state", "global")
        density_analysis.input_coordinate_file = ProtocolPath("output_coordinate_file", production_simulation.id)
        density_analysis.input_trajectory_path = ProtocolPath("trajectory_file_path", production_simulation.id)
        density_analysis.output_to_store = ProtocolPath("output_density", production_simulation.id)
        density_analysis.conditional_group = "density_converged"
        schema.protocol_schemas.append(density_analysis)

        return schema

# Define the substance from rdkit
benzene_smiles = "c1ccccc1"
benzene_mol = Chem.AddHs(Chem.MolFromSmiles(benzene_smiles))
rdDistGeom.EmbedMolecule(benzene_mol)
smiles = Chem.MolToSmiles(benzene_mol)
benzene = Substance()
benzene.add_component(Component(smiles=smiles), MoleFraction(1.0))

# Define the thermodynamic state
thermodynamic_state = ThermodynamicState(
    temperature=298.15 * unit.kelvin,
    pressure=1.0 * unit.atmosphere
)

# Define the force field
forcefield = ForceField("openff_unconstrained-2.1.0.offxml")
forcefield_path = "openff_unconstrained-2.1.0.offxml"

# Define the workflow
schema = CustomDensityWorkflow.default_simulation_schema()
workflow = Workflow()
workflow.schema = schema

# Define evaluator client
client = EvaluatorClient()

# Define request options
options = RequestOptions()
options.calculation_layers = [workflow]

# Add global variables
workflow.schema.global_protocols["force_field_path"] = forcefield_path
workflow.schema.global_protocols["coordinate_file_path"] = "build_coordinates"

# Create and submit the estimation request
request = client.request_estimate(
    properties=[CustomDensityWorkflow()],
    substances=[benzene],
    thermodynamic_states=[thermodynamic_state],
    options=options,
)

# Wait for results
results = request.results()
print(results)


TypeError: The value must be a unit-wrapped integer, float or numpy array.

In [ ]:
schema = WorkflowSchema()

# Step 1: Build coordinates
build_coordinates = BuildCoordinatesPackmol("build_coordinates")
build_coordinates.max_molecules = 1000
build_coordinates.mass_density = 0.95 * unit.grams / unit.milliliters
# schema.protocol_schemas.append(build_coordinates)

In [ ]:
schema.get_attributes()

['protocol_schemas',
 'protocol_replicators',
 'final_value_source',
 'outputs_to_store']

In [ ]:
schema.protocol_schemas.append(build_coordinates)

In [ ]:
import os
from rdkit import Chem
from rdkit.Chem import rdDistGeom
from openff.evaluator import unit
from openff.evaluator.client import EvaluatorClient, RequestOptions
from openff.evaluator.forcefield import ForceFieldSource
from openff.evaluator.workflow import Workflow
from openff.evaluator.workflow.utils import ProtocolPath

from openff.evaluator.workflow.schemas import WorkflowSchema
from openff.evaluator.properties.density import Density
from openff.evaluator.protocols.coordinates import BuildCoordinatesPackmol
from openff.evaluator.protocols.forcefield import BuildSmirnoffSystem
from openff.evaluator.protocols.openmm import OpenMMSimulation, OpenMMEnergyMinimisation

from openff.evaluator.utils.observables import ObservableType, ObservableArray
from openff.evaluator.substances import Substance, Component, MoleFraction
from openff.evaluator.thermodynamics import ThermodynamicState
from openff.toolkit.typing.engines.smirnoff import ForceField

# Define the custom density workflow
class CustomDensityWorkflow(Density):
    @staticmethod
    def default_simulation_schema(replicas: int = 3) -> WorkflowSchema:
        schema = WorkflowSchema()

        # Step 1: Build coordinates
        build_coordinates = BuildCoordinatesPackmol("build_coordinates")
        build_coordinates.max_molecules = 1000
        build_coordinates.mass_density = 0.95 * unit.grams / unit.milliliters
        schema.protocol_schemas.append(build_coordinates)

        # Step 2: Assign parameters
        assign_parameters = BuildSmirnoffSystem("assign_parameters")
        assign_parameters.force_field_path = ProtocolPath("force_field_path", "global")
        assign_parameters.coordinate_file_path = ProtocolPath("coordinate_file_path", build_coordinates.id)
        schema.protocol_schemas.append(assign_parameters)

        # Step 3: Energy minimization
        energy_minimization = OpenMMEnergyMinimisation("energy_minimisation")
        energy_minimization.input_coordinate_file = ProtocolPath("coordinate_file_path", assign_parameters.id)
        energy_minimization.input_force_field = ProtocolPath("parameterized_system", assign_parameters.id)
        schema.protocol_schemas.append(energy_minimization)

        # Step 4: Equilibration simulation
        equilibration_simulation = OpenMMSimulation("equilibration_simulation")
        equilibration_simulation.input_coordinate_file = ProtocolPath("output_coordinate_file", energy_minimization.id)
        equilibration_simulation.input_force_field = ProtocolPath("parameterized_system", assign_parameters.id)
        equilibration_simulation.steps = 100000
        schema.protocol_schemas.append(equilibration_simulation)

        return schema

def run_short_simulation(client, workflow, benzene, thermodynamic_state, forcefield_path, steps):
    # Define the simulation protocol
    production_simulation = OpenMMSimulation("short_production_simulation")
    production_simulation.input_coordinate_file = ProtocolPath("output_coordinate_file", "energy_minimisation")
    production_simulation.input_force_field = ProtocolPath("parameterized_system", "assign_parameters")
    production_simulation.steps = steps

    workflow.schema.protocol_schemas.append(production_simulation)
    
    # Add global variables
    workflow.schema.global_protocols["force_field_path"] = forcefield_path
    workflow.schema.global_protocols["coordinate_file_path"] = "build_coordinates"

    # Create and submit the estimation request
    request = client.request_estimate(
        properties=[workflow],
        substances=[benzene],
        thermodynamic_states=[thermodynamic_state],
        options=RequestOptions()
    )
    
    # Wait for results
    results = request.results()
    
    return results[0].value

# Define the substance from rdkit
benzene_smiles = "c1ccccc1"
benzene_mol = Chem.AddHs(Chem.MolFromSmiles(benzene_smiles))
rdDistGeom.EmbedMolecule(benzene_mol)
smiles = Chem.MolToSmiles(benzene_mol)
benzene = Substance()
benzene.add_component(Component(smiles=smiles), MoleFraction(1.0))

# Define the thermodynamic state
thermodynamic_state = ThermodynamicState(
    temperature=298.15 * unit.kelvin,
    pressure=1.0 * unit.atmosphere
)

# Define the force field
forcefield = ForceField("openff_unconstrained-2.1.0.offxml")
forcefield_path = "openff_unconstrained-2.1.0.offxml"

# Define the evaluator client
client = EvaluatorClient()

# Initial workflow
schema = CustomDensityWorkflow.default_simulation_schema()
workflow = Workflow()
workflow.schema = schema

# Run short simulations until convergence
convergence_tolerance = 0.01  # g/mL
max_iterations = 10
steps = 10000  # Adjusted for shorter simulation time

previous_density = None
for iteration in range(max_iterations):
    current_density = run_short_simulation(client, workflow, benzene, thermodynamic_state, forcefield_path, steps)
    if previous_density is not None:
        if abs(current_density - previous_density) < convergence_tolerance:
            print(f"Density has converged: {current_density} g/mL")
            break
    previous_density = current_density
    print(f"Iteration {iteration + 1}: Current density = {current_density} g/mL")

# Run final production simulation if converged
if abs(current_density - previous_density) < convergence_tolerance:
    # Final production run
    final_production_simulation = OpenMMSimulation("final_production_simulation")
    final_production_simulation.input_coordinate_file = ProtocolPath("output_coordinate_file", "equilibration_simulation")
    final_production_simulation.input_force_field = ProtocolPath("parameterized_system", "assign_parameters")
    final_production_simulation.steps = 50000  # 0.1 ns run for testing

    workflow.schema.protocol_schemas.append(final_production_simulation)

    # Create and submit the estimation request for final production run
    request = client.request_estimate(
        properties=[workflow],
        substances=[benzene],
        thermodynamic_states=[thermodynamic_state],
        options=RequestOptions()
    )

    # Wait for results
    results = request.results()
    final_density = results[0].value
    print(f"Final density after production run: {final_density} g/mL")
else:
    print("Density did not converge within the maximum number of iterations.")


TypeError: Workflow.__init__() missing 1 required positional argument: 'global_metadata'

In [ ]:

class CustomDensityWorkflow(Density):
    @staticmethod
    def default_simulation_schema(replicas: int = 3) -> WorkflowSchema:
        schema = WorkflowSchema()

        # Step 1: Build coordinates
        build_coordinates = BuildCoordinatesPackmol("build_coordinates")
        build_coordinates.max_molecules = 1000
        build_coordinates.mass_density = 0.95 * unit.grams / unit.milliliters
        schema.protocol_schemas.append(build_coordinates)

        # Step 2: Assign parameters
        assign_parameters = BuildSmirnoffSystem("assign_parameters")
        assign_parameters.force_field_path = ProtocolPath("force_field_path", "global")
        assign_parameters.coordinate_file_path = ProtocolPath("coordinate_file_path", build_coordinates.id)
        schema.protocol_schemas.append(assign_parameters)

        # Step 3: Energy minimization
        energy_minimization = OpenMMEnergyMinimisation("energy_minimisation")
        energy_minimization.input_coordinate_file = ProtocolPath("coordinate_file_path", assign_parameters.id)
        energy_minimization.input_force_field = ProtocolPath("parameterized_system", assign_parameters.id)
        schema.protocol_schemas.append(energy_minimization)

        # Step 4: Equilibration simulation
        equilibration_simulation = OpenMMSimulation("equilibration_simulation")
        equilibration_simulation.input_coordinate_file = ProtocolPath("output_coordinate_file", energy_minimization.id)
        equilibration_simulation.input_force_field = ProtocolPath("parameterized_system", assign_parameters.id)
        equilibration_simulation.steps = 100000
        schema.protocol_schemas.append(equilibration_simulation)

        return schema

def run_short_simulation(client, workflow, benzene, thermodynamic_state, forcefield_path, steps):
    # Define the simulation protocol
    production_simulation = OpenMMSimulation("short_production_simulation")
    production_simulation.input_coordinate_file = ProtocolPath("output_coordinate_file", "energy_minimisation")
    production_simulation.input_force_field = ProtocolPath("parameterized_system", "assign_parameters")
    production_simulation.steps = steps

    workflow.schema.protocol_schemas.append(production_simulation)
    
    # Add global variables
    workflow.schema.global_protocols["force_field_path"] = forcefield_path
    workflow.schema.global_protocols["coordinate_file_path"] = "build_coordinates"

    # Create and submit the estimation request
    request = client.request_estimate(
        properties=[workflow],
        substances=[benzene],
        thermodynamic_states=[thermodynamic_state],
        options=RequestOptions()
    )
    
    # Wait for results
    results = request.results()
    
    return results[0].value

# Define the substance from rdkit
benzene_smiles = "c1ccccc1"
benzene_mol = Chem.AddHs(Chem.MolFromSmiles(benzene_smiles))
rdDistGeom.EmbedMolecule(benzene_mol)
smiles = Chem.MolToSmiles(benzene_mol)
benzene = Substance()
benzene.add_component(Component(smiles=smiles), MoleFraction(1.0))

# Define the thermodynamic state
thermodynamic_state = ThermodynamicState(
    temperature=298.15 * unit.kelvin,
    pressure=1.0 * unit.atmosphere
)

# Define the force field
forcefield = ForceField("openff_unconstrained-2.1.0.offxml")
forcefield_path = "openff_unconstrained-2.1.0.offxml"

# Define the evaluator client
client = EvaluatorClient()
# Initial workflow
target_uncertainty = 0.1 * unit.grams / unit.milliliters
schema = CustomDensityWorkflow.default_simulation_schema()
global_metadata = {
            "thermodynamic_state": Density.thermodynamic_state,
            # "substance": Density.substance,
            # "components": benzene,
            # "target_uncertainty": target_uncertainty,
            # "force_field_path": forcefield_path,
        }
workflow = Workflow.from_schema(schema, metadata=global_metadata)

# Run short simulations until convergence
convergence_tolerance = 0.01  # g/mL
max_iterations = 10
steps = 10000  # Adjusted for shorter simulation time

previous_density = None
for iteration in range(max_iterations):
    current_density = run_short_simulation(client, workflow, benzene, thermodynamic_state, forcefield_path, steps)
    if previous_density is not None:
        if abs(current_density - previous_density) < convergence_tolerance:
            print(f"Density has converged: {current_density} g/mL")
            break
    previous_density = current_density
    print(f"Iteration {iteration + 1}: Current density = {current_density} g/mL")

# Run final production simulation if converged
if abs(current_density - previous_density) < convergence_tolerance:
    # Final production run
    final_production_simulation = OpenMMSimulation("final_production_simulation")
    final_production_simulation.input_coordinate_file = ProtocolPath("output_coordinate_file", "equilibration_simulation")
    final_production_simulation.input_force_field = ProtocolPath("parameterized_system", "assign_parameters")
    final_production_simulation.steps = 50000  # 0.1 ns run for testing

    workflow.schema.protocol_schemas.append(final_production_simulation)

    # Create and submit the estimation request for final production run
    request = client.request_estimate(
        properties=[workflow],
        substances=[benzene],
        thermodynamic_states=[thermodynamic_state],
        options=RequestOptions()
    )

    # Wait for results
    results = request.results()
    final_density = results[0].value
    print(f"Final density after production run: {final_density} g/mL")
else:
    print("Density did not converge within the maximum number of iterations.")

ImportError: cannot import name 'ProtocolPath' from 'openff.evaluator.workflow' (/localhome/cschiebroek/MDFP_VP/mdfptools/carl/openff-evaluator/openff/evaluator/workflow/__init__.py)

In [ ]:


class CustomDensityWorkflow(Density):
    @staticmethod
    def default_simulation_schema(replicas: int = 3) -> WorkflowSchema:
        schema = WorkflowSchema()

        # Step 1: Build coordinates
        build_coordinates = BuildCoordinatesPackmol("build_coordinates")
        build_coordinates.max_molecules = 1000
        build_coordinates.mass_density = 0.95 * unit.grams / unit.milliliters
        schema.protocol_schemas.append(build_coordinates.schema)

        # Step 2: Assign parameters
        assign_parameters = BuildSmirnoffSystem("assign_parameters")
        assign_parameters.force_field_path = ProtocolPath("force_field_path", "global")
        assign_parameters.coordinate_file_path = ProtocolPath("coordinate_file_path", build_coordinates.id)
        schema.protocol_schemas.append(assign_parameters.schema)

        # Step 3: Energy minimization
        energy_minimization = OpenMMEnergyMinimisation("energy_minimisation")
        energy_minimization.input_coordinate_file = ProtocolPath("coordinate_file_path", assign_parameters.id)
        energy_minimization.input_force_field = ProtocolPath("parameterized_system", assign_parameters.id)
        schema.protocol_schemas.append(energy_minimization.schema)

        # Step 4: Equilibration simulation
        equilibration_simulation = OpenMMSimulation("equilibration_simulation")
        equilibration_simulation.input_coordinate_file = ProtocolPath("output_coordinate_file", energy_minimization.id)
        equilibration_simulation.input_force_field = ProtocolPath("parameterized_system", assign_parameters.id)
        equilibration_simulation.steps = 100000
        schema.protocol_schemas.append(equilibration_simulation.schema)

        #add global variables
        schema.metadata["force_field_path"] = forcefield_path

        return schema

def run_short_simulation(client, schema, benzene, thermodynamic_state, forcefield_path, steps):
    # Define the simulation protocol
    production_simulation = OpenMMSimulation("short_production_simulation")
    production_simulation.input_coordinate_file = ProtocolPath("output_coordinate_file", "energy_minimisation")
    production_simulation.input_force_field = ProtocolPath("parameterized_system", "assign_parameters")
    production_simulation.steps = steps

    schema.protocol_schemas.append(production_simulation.schema)
    
    # Create the workflow from schema
    workflow = Workflow.from_schema(schema, metadata=None)

    # Create and submit the estimation request
    request = client.request_estimate(
        properties=[workflow],
        substances=[benzene],
        thermodynamic_states=[thermodynamic_state],
        options=RequestOptions()
    )
    
    # Wait for results
    results = request.results()
    
    return results[0].value

# Define the substance from rdkit
benzene_smiles = "c1ccccc1"
benzene_mol = Chem.AddHs(Chem.MolFromSmiles(benzene_smiles))
rdDistGeom.EmbedMolecule(benzene_mol)
smiles = Chem.MolToSmiles(benzene_mol)
benzene = Substance()
benzene.add_component(Component(smiles=smiles), MoleFraction(1.0))

# Define the thermodynamic state
thermodynamic_state = ThermodynamicState(
    temperature=298.15 * unit.kelvin,
    pressure=1.0 * unit.atmosphere
)

# Define the force field
forcefield = ForceField("openff_unconstrained-2.1.0.offxml")
forcefield_path = "openff_unconstrained-2.1.0.offxml"

# Define the evaluator client
client = EvaluatorClient()
# Initial workflow
target_uncertainty = 0.1 * unit.grams / unit.milliliters
schema = CustomDensityWorkflow.default_simulation_schema()
global_metadata = {
            "substance": Density.substance,
        }

# Run short simulations until convergence
convergence_tolerance = 0.01  # g/mL
max_iterations = 10
steps = 10000  # Adjusted for shorter simulation time
schema.metadata = global_metadata
previous_density = None
for iteration in range(max_iterations):
    current_density = run_short_simulation(client, schema, benzene, thermodynamic_state, forcefield_path, steps)
    if previous_density is not None:
        if abs(current_density - previous_density) < convergence_tolerance:
            print(f"Density has converged: {current_density} g/mL")
            break
    previous_density = current_density
    print(f"Iteration {iteration + 1}: Current density = {current_density} g/mL")

# Run final production simulation if converged
if abs(current_density - previous_density) < convergence_tolerance:
    # Final production run
    final_production_simulation = OpenMMSimulation("final_production_simulation")
    final_production_simulation.input_coordinate_file = ProtocolPath("output_coordinate_file", "equilibration_simulation")
    final_production_simulation.input_force_field = ProtocolPath("parameterized_system", "assign_parameters")
    final_production_simulation.steps = 50000  # 0.1 ns run for testing

    schema.protocol_schemas.append(final_production_simulation.schema)

    # Create the workflow from schema
    workflow = Workflow.from_schema(schema, metadata=global_metadata)

    # Create and submit the estimation request for final production run
    request = client.request_estimate(
        properties=[workflow],
        substances=[benzene],
        thermodynamic_states=[thermodynamic_state],
        options=RequestOptions()
    )

    # Wait for results
    results = request.results()
    final_density = results[0].value
    print(f"Final density after production run: {final_density} g/mL")
else:
    print("Density did not converge within the maximum number of iterations.")


AttributeError: 'WorkflowSchema' object has no attribute 'metadata'

In [1]:
import os
from rdkit import Chem
from rdkit.Chem import rdDistGeom
from openff.evaluator import unit
from openff.evaluator.client import EvaluatorClient, RequestOptions
from openff.evaluator.forcefield import ForceFieldSource
from openff.evaluator.workflow import Workflow
from openff.evaluator.workflow.utils import ProtocolPath

from openff.evaluator.workflow.schemas import WorkflowSchema
from openff.evaluator.properties.density import Density
from openff.evaluator.protocols.coordinates import BuildCoordinatesPackmol
from openff.evaluator.protocols.forcefield import BuildSmirnoffSystem
from openff.evaluator.protocols.openmm import OpenMMSimulation, OpenMMEnergyMinimisation

from openff.evaluator.utils.observables import ObservableType, ObservableArray
from openff.evaluator.substances import Substance, Component, MoleFraction
from openff.evaluator.thermodynamics import ThermodynamicState
from openff.toolkit.typing.engines.smirnoff import ForceField


Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [4]:
benzene_smiles = "c1ccccc1"
benzene_mol = Chem.AddHs(Chem.MolFromSmiles(benzene_smiles))
rdDistGeom.EmbedMolecule(benzene_mol)
smiles = Chem.MolToSmiles(benzene_mol)
substance = Substance()
substance.add_component(Component(smiles=smiles), MoleFraction(1.0))
metadata = {"substance": substance}

# Step 1: Build coordinates
build_coordinates = BuildCoordinatesPackmol("build_coordinates")
build_coordinates.allow_merging = True
build_coordinates.count_exact_amount = True
build_coordinates.mass_density = 0.95 * unit.grams / unit.milliliters
build_coordinates.box_aspect_ratio = [1.0, 1.0, 1.0]
build_coordinates.substance = ProtocolPath("substance", "global")
build_coordinates.tolerance = 2.0 * unit.angstrom
build_coordinates.verbose_packmol = False
build_coordinates.retain_packmol_files = False


# Step 2: Assign parameters
assign_parameters = BuildSmirnoffSystem("assign_parameters")
assign_parameters.allow_merging = True
assign_parameters.force_field_path = ProtocolPath("force_field_path", "global")
assign_parameters.coordinate_file_path = ProtocolPath("coordinate_file_path", build_coordinates.id)
assign_parameters.substance = ProtocolPath("substance", "global")

# Step 3: Energy minimization
energy_minimization = OpenMMEnergyMinimisation("energy_minimisation")
energy_minimization.allow_merging = True
energy_minimization.input_coordinate_file = ProtocolPath("coordinate_file_path", build_coordinates.id)
energy_minimization.parameterized_system = ProtocolPath("parameterized_system", assign_parameters.id)
energy_minimization.tolerance = 10.0 * unit.kilojoules_per_mole / unit.nanometer
energy_minimization.enable_pbc = True
energy_minimization.max_iterations = 0

# Step 4: Equilibration simulation
equilibration_simulation = OpenMMSimulation("equilibration_simulation")
equilibration_simulation.allow_merging = True
equilibration_simulation.steps_per_iteration = 100000
equilibration_simulation.total_number_of_iterations = 1
equilibration_simulation.output_frequency = 5000
equilibration_simulation.checkpoint_frequency = 10
equilibration_simulation.timestep = 2.0 * unit.femtosecond
equilibration_simulation.thermodynamic_state = ProtocolPath("thermodynamic_state", "global")
equilibration_simulation.thermostat_friction = 1.0 / unit.picosecond
equilibration_simulation.input_coordinate_file = ProtocolPath("output_coordinate_file", energy_minimization.id)
equilibration_simulation.parameterized_system = ProtocolPath("parameterized_system", assign_parameters.id)
equilibration_simulation.enable_pbc = True
equilibration_simulation.allow_gpu_platforms = True
equilibration_simulation.high_precision = False

schema = WorkflowSchema()
schema.protocol_schemas = [build_coordinates.schema, assign_parameters.schema, energy_minimization.schema, equilibration_simulation.schema]

metadata = {
    "substance": substance,
    "thermodynamic_state": ThermodynamicState(
        temperature=298.15 * unit.kelvin,
        pressure=1.0 * unit.atmosphere
    ),
    "force_field_path": "openff_unconstrained-2.1.0.offxml"
}
# Create the workflow from schema
workflow = Workflow.from_schema(schema, metadata=metadata)

In [ ]:
#now run production simulation for 5ns
production_simulation = OpenMMSimulation("production_simulation")
production_simulation.allow_merging = True
production_simulation.steps_per_iteration = 100000
production_simulation.total_number_of_iterations = 50
production_simulation.output_frequency = 5000
production_simulation.checkpoint_frequency = 10
production_simulation.timestep = 2.0 * unit.femtosecond
production_simulation.thermodynamic_state = ProtocolPath("thermodynamic_state", "global")
production_simulation.thermostat_friction = 1.0 / unit.picosecond
production_simulation.input_coordinate_file = ProtocolPath("output_coordinate_file", equilibration_simulation.id)
production_simulation.parameterized_system = ProtocolPath("parameterized_system", assign_parameters.id)
production_simulation.enable_pbc = True
production_simulation.allow_gpu_platforms = True
production_simulation.high_precision = False
schema.protocol_schemas.append(production_simulation.schema)

In [5]:
results = workflow.execute()

In [22]:
density_workflow = Density.default_simulation_schema()
density_workflow.workflow_schema.protocol_schemas[5].inputs

{'.allow_merging': True,
 '.time_series_statistics': <ProtocolPath full_path=conditional_group/average_density.time_series_statistics>,
 '.input_coordinate_file': <ProtocolPath full_path=conditional_group/production_simulation.output_coordinate_file>,
 '.input_trajectory_path': <ProtocolPath full_path=conditional_group/production_simulation.trajectory_file_path>}

In [ ]:
# Step 6: Analysis step to check if density has converged
density_analysis = AverageObservable("density_analysis")
density_analysis.observable = ObservableType.Density
density_analysis.thermodynamic_state = ProtocolPath("thermodynamic_state", "global")
density_analysis.input_coordinate_file = ProtocolPath("output_coordinate_file", production_simulation.id)
density_analysis.input_trajectory_path = ProtocolPath("trajectory_file_path", production_simulation.id)
density_analysis.output_to_store = ProtocolPath("output_density", production_simulation.id)
density_analysis.conditional_group = "density_converged"
schema.protocol_schemas.append(density_analysis)

# Step 7: Final production run after density convergence
final_production = OpenMMSimulation("final_production_simulation")
final_production.input_coordinate_file = ProtocolPath("output_coordinate_file", equilibration_simulation.id)
final_production.input_force_field = ProtocolPath("parameterized_system", assign_parameters.id)
final_production.steps = 50000  # 0.1 ns run for testing
final_production.conditional_group = "density_converged"
schema.protocol_schemas.append(final_production)

# Create the workflow from schema
workflow = Workflow.from_schema(schema, metadata=metadata)




In [40]:
from openff.evaluator.protocols.analysis import AverageObservable
from openff.evaluator.utils.observables import ObservableType, ObservableArray
from  openff.evaluator.protocols.groups import ConditionalGroup

In [38]:
# Step 1: Convert SMILES to a molecule and add hydrogens
benzene_smiles = "c1ccccc1"
benzene_mol = Chem.AddHs(Chem.MolFromSmiles(benzene_smiles))
rdDistGeom.EmbedMolecule(benzene_mol)

# Convert molecule to SMILES again (to ensure it's correctly hydrogenated)
smiles = Chem.MolToSmiles(benzene_mol)
substance = Substance()
substance.add_component(Component(smiles=smiles), MoleFraction(1.0))

# Step 2: Build coordinates
build_coordinates = BuildCoordinatesPackmol("build_coordinates")
build_coordinates.allow_merging = True
build_coordinates.count_exact_amount = True
build_coordinates.mass_density = 0.95 * unit.grams / unit.milliliters
build_coordinates.box_aspect_ratio = [1.0, 1.0, 1.0]
build_coordinates.substance = ProtocolPath("substance", "global")
build_coordinates.tolerance = 2.0 * unit.angstrom
build_coordinates.verbose_packmol = False
build_coordinates.retain_packmol_files = False

# Step 3: Assign parameters
assign_parameters = BuildSmirnoffSystem("assign_parameters")
assign_parameters.allow_merging = True
assign_parameters.force_field_path = ProtocolPath("force_field_path", "global")
assign_parameters.coordinate_file_path = ProtocolPath("coordinate_file_path", build_coordinates.id)
assign_parameters.substance = ProtocolPath("substance", "global")

# Step 4: Energy minimization
energy_minimization = OpenMMEnergyMinimisation("energy_minimisation")
energy_minimization.allow_merging = True
energy_minimization.input_coordinate_file = ProtocolPath("coordinate_file_path", build_coordinates.id)
energy_minimization.parameterized_system = ProtocolPath("parameterized_system", assign_parameters.id)
energy_minimization.tolerance = 10.0 * unit.kilojoules_per_mole / unit.nanometer
energy_minimization.enable_pbc = True
energy_minimization.max_iterations = 0

# Step 5: Equilibration simulation
equilibration_simulation = OpenMMSimulation("equilibration_simulation")
equilibration_simulation.allow_merging = True
equilibration_simulation.steps_per_iteration = 100000
equilibration_simulation.total_number_of_iterations = 1
equilibration_simulation.output_frequency = 5000
equilibration_simulation.checkpoint_frequency = 10
equilibration_simulation.timestep = 2.0 * unit.femtosecond
equilibration_simulation.thermodynamic_state = ProtocolPath("thermodynamic_state", "global")
equilibration_simulation.thermostat_friction = 1.0 / unit.picosecond
equilibration_simulation.input_coordinate_file = ProtocolPath("output_coordinate_file", energy_minimization.id)
equilibration_simulation.parameterized_system = ProtocolPath("parameterized_system", assign_parameters.id)
equilibration_simulation.enable_pbc = True
equilibration_simulation.allow_gpu_platforms = True
equilibration_simulation.high_precision = False


# Step 6: Analysis step to check if density has converged
density_analysis = AverageObservable("density_analysis")
density_analysis.observable = ObservableType.Density
density_analysis.thermodynamic_state = ProtocolPath("thermodynamic_state", "global")
density_analysis.input_coordinate_file = ProtocolPath("output_coordinate_file", equilibration_simulation.id)
density_analysis.input_trajectory_path = ProtocolPath("trajectory_file_path", equilibration_simulation.id)
density_analysis.output_observable = "density"

# Step 7: Define a conditional group for final production simulation
final_production = OpenMMSimulation("final_production_simulation")
final_production.input_coordinate_file = ProtocolPath("output_coordinate_file", equilibration_simulation.id)
final_production.input_force_field = ProtocolPath("parameterized_system", assign_parameters.id)
final_production.steps = 50000  # 0.1 ns run for testing

conditional_group = ConditionalGroup("conditional_group")
conditional_group.add_protocols(final_production, density_analysis)

# Set up a condition which will check if the density uncertainty is less than some threshold.
condition = ConditionalGroup.Condition()
condition.condition_type = ConditionalGroup.Condition.Type.LessThan
condition.right_hand_value = 0.5 * unit.grams / unit.milliliter
condition.left_hand_value = ProtocolPath("value.error", conditional_group.id, density_analysis.id)

# Add the condition.
conditional_group.add_condition(condition)

# Step 8: Define schema and metadata
schema = WorkflowSchema()
schema.protocol_schemas = [
    build_coordinates.schema,
    assign_parameters.schema,
    energy_minimization.schema,
    equilibration_simulation.schema,
    density_analysis.schema,
    conditional_group.schema
]

metadata = {
    "substance": substance,
    "thermodynamic_state": ThermodynamicState(
        temperature=298.15 * unit.kelvin,
        pressure=1.0 * unit.atmosphere
    ),
    "force_field_path": "openff_unconstrained-2.1.0.offxml"
}

# Step 9: Create the workflow from schema
workflow = Workflow.from_schema(schema, metadata=metadata)

# Step 10: Execute the workflow
workflow.execute()


ValueError: The observable attribute can only accept values of type <class 'openff.evaluator.utils.observables.ObservableArray'>

In [93]:


# Step 1: Convert SMILES to a molecule and add hydrogens
benzene_smiles = "c1ccccc1"
benzene_mol = Chem.AddHs(Chem.MolFromSmiles(benzene_smiles))
rdDistGeom.EmbedMolecule(benzene_mol)

# Convert molecule to SMILES again (to ensure it's correctly hydrogenated)
smiles = Chem.MolToSmiles(benzene_mol)
substance = Substance()
substance.add_component(Component(smiles=smiles), MoleFraction(1.0))

# Step 2: Build coordinates
build_coordinates = BuildCoordinatesPackmol("build_coordinates")
build_coordinates.allow_merging = True
build_coordinates.count_exact_amount = True
build_coordinates.mass_density = 0.95 * unit.grams / unit.milliliters
build_coordinates.box_aspect_ratio = [1.0, 1.0, 1.0]
build_coordinates.substance = ProtocolPath("substance", "global")
build_coordinates.tolerance = 2.0 * unit.angstrom
build_coordinates.verbose_packmol = False
build_coordinates.retain_packmol_files = False

# Step 3: Assign parameters
assign_parameters = BuildSmirnoffSystem("assign_parameters")
assign_parameters.allow_merging = True
assign_parameters.force_field_path = ProtocolPath("force_field_path", "global")
assign_parameters.coordinate_file_path = ProtocolPath("coordinate_file_path", build_coordinates.id)
assign_parameters.substance = ProtocolPath("substance", "global")

# Step 4: Energy minimization
energy_minimization = OpenMMEnergyMinimisation("energy_minimisation")
energy_minimization.allow_merging = True
energy_minimization.input_coordinate_file = ProtocolPath("coordinate_file_path", build_coordinates.id)
energy_minimization.parameterized_system = ProtocolPath("parameterized_system", assign_parameters.id)
energy_minimization.tolerance = 10.0 * unit.kilojoules_per_mole / unit.nanometer
energy_minimization.enable_pbc = True
energy_minimization.max_iterations = 0

# Step 5: Equilibration simulation
equilibration_simulation = OpenMMSimulation("equilibration_simulation")
equilibration_simulation.allow_merging = True
equilibration_simulation.steps_per_iteration = 100000
equilibration_simulation.total_number_of_iterations = 1
equilibration_simulation.output_frequency = 5000
equilibration_simulation.checkpoint_frequency = 10
equilibration_simulation.timestep = 2.0 * unit.femtosecond
equilibration_simulation.thermodynamic_state = ProtocolPath("thermodynamic_state", "global")
equilibration_simulation.thermostat_friction = 1.0 / unit.picosecond
equilibration_simulation.input_coordinate_file = ProtocolPath("output_coordinate_file", energy_minimization.id)
equilibration_simulation.parameterized_system = ProtocolPath("parameterized_system", assign_parameters.id)
equilibration_simulation.enable_pbc = True
equilibration_simulation.allow_gpu_platforms = True
equilibration_simulation.high_precision = False

# Step 6: Analysis step to check if density fluctuation has converged
density_analysis = AverageObservable("density_analysis")
density_analysis.observable = ProtocolPath("observables.Density", equilibration_simulation.id)
density_analysis.thermodynamic_state = ProtocolPath("thermodynamic_state", "global")
density_analysis.input_coordinate_file = ProtocolPath("output_coordinate_file", equilibration_simulation.id)
density_analysis.input_trajectory_path = ProtocolPath("trajectory_file_path", equilibration_simulation.id)
density_analysis.output_observable = "density_fluctuation"

# Step 7: Define a conditional group for final production simulation
final_production = OpenMMSimulation("final_production_simulation")
final_production.input_coordinate_file = ProtocolPath("output_coordinate_file", equilibration_simulation.id)
final_production.input_force_field = ProtocolPath("parameterized_system", assign_parameters.id)
final_production.steps = 50000  # 0.1 ns run for testing
final_production.thermodynamic_state = ProtocolPath("thermodynamic_state", "global")
final_production.parameterized_system = ProtocolPath("parameterized_system", assign_parameters.id)

conditional_group = ConditionalGroup("conditional_group")
conditional_group.add_protocols(final_production, density_analysis)

# Set up a condition which will check if the density fluctuation is less than some threshold.
condition = ConditionalGroup.Condition()
condition.condition_type = ConditionalGroup.Condition.Type.LessThan
condition.right_hand_value = 0.5 * unit.grams / unit.milliliter  # Threshold for fluctuation
condition.left_hand_value = ProtocolPath("value.std_error", density_analysis.id)

# Add the condition.
conditional_group.add_condition(condition)

# Step 8: Define schema and metadata
schema = WorkflowSchema()
schema.protocol_schemas = [
    build_coordinates.schema,
    assign_parameters.schema,
    energy_minimization.schema,
    equilibration_simulation.schema,
    conditional_group.schema
]

metadata = {
    "substance": substance,
    "thermodynamic_state": ThermodynamicState(
        temperature=298.15 * unit.kelvin,
        pressure=1.0 * unit.atmosphere
    ),
    "force_field_path": "openff_unconstrained-2.1.0.offxml"
}

# Step 9: Create the workflow from schema
workflow = Workflow.from_schema(schema, metadata=metadata)

# Step 10: Execute the workflow
workflow.execute()


ValueError: The conditional_group protocol tries to take input from a non-existent protocol: density_analysis.value.std_error

In [82]:
density_analysis.get_attributes()

['id',
 'allow_merging',
 'bootstrap_iterations',
 'bootstrap_sample_size',
 'thermodynamic_state',
 'potential_energies',
 'value',
 'time_series_statistics',
 'observable',
 'divisor']

In [87]:
density_analysis.time_series_statistics

In [58]:
#ValueError: Several protocols in the schema have the same id: ['density_analysis']. This is currently unsupported due to issues with merging two graphs which contain duplicate ids. print out all ids
for protocol in schema.protocol_schemas:
    print(protocol.id)

build_coordinates
assign_parameters
energy_minimisation
equilibration_simulation
density_analysis
conditional_group


In [66]:
schema = WorkflowSchema()
schema.protocol_schemas = [
    build_coordinates.schema,
    assign_parameters.schema,
    energy_minimization.schema,
    equilibration_simulation.schema,
    density_analysis.schema,
    conditional_group.schema
]
for protocol in schema.protocol_schemas:
    print(protocol.id)

build_coordinates
assign_parameters
energy_minimisation
equilibration_simulation
density_analysis
conditional_group


In [70]:
schema.protocol_schemas[-1].protocol_schemas

{'final_production_simulation': <openff.evaluator.workflow.schemas.ProtocolSchema at 0x7d42d9fc7fd0>,
 'density_analysis': <openff.evaluator.workflow.schemas.ProtocolSchema at 0x7d430cf53850>}

In [36]:
equilibration_simulation.observables["Density"]

TypeError: 'UndefinedAttribute' object is not subscriptable

In [97]:
# Step 1: Convert SMILES to a molecule and add hydrogens
benzene_smiles = "c1ccccc1"
benzene_mol = Chem.AddHs(Chem.MolFromSmiles(benzene_smiles))
rdDistGeom.EmbedMolecule(benzene_mol)

# Convert molecule to SMILES again (to ensure it's correctly hydrogenated)
smiles = Chem.MolToSmiles(benzene_mol)
substance = Substance()
substance.add_component(Component(smiles=smiles), MoleFraction(1.0))

# Step 2: Build coordinates
build_coordinates = BuildCoordinatesPackmol("build_coordinates")
build_coordinates.allow_merging = True
build_coordinates.count_exact_amount = True
build_coordinates.mass_density = 0.95 * unit.grams / unit.milliliters
build_coordinates.box_aspect_ratio = [1.0, 1.0, 1.0]
build_coordinates.substance = ProtocolPath("substance", "global")
build_coordinates.tolerance = 2.0 * unit.angstrom
build_coordinates.verbose_packmol = False
build_coordinates.retain_packmol_files = False

# Step 3: Assign parameters
assign_parameters = BuildSmirnoffSystem("assign_parameters")
assign_parameters.allow_merging = True
assign_parameters.force_field_path = ProtocolPath("force_field_path", "global")
assign_parameters.coordinate_file_path = ProtocolPath("coordinate_file_path", build_coordinates.id)
assign_parameters.substance = ProtocolPath("substance", "global")

# Step 4: Energy minimization
energy_minimization = OpenMMEnergyMinimisation("energy_minimisation")
energy_minimization.allow_merging = True
energy_minimization.input_coordinate_file = ProtocolPath("coordinate_file_path", build_coordinates.id)
energy_minimization.parameterized_system = ProtocolPath("parameterized_system", assign_parameters.id)
energy_minimization.tolerance = 10.0 * unit.kilojoules_per_mole / unit.nanometer
energy_minimization.enable_pbc = True
energy_minimization.max_iterations = 0

# Step 5: Equilibration simulation
equilibration_simulation = OpenMMSimulation("equilibration_simulation")
equilibration_simulation.allow_merging = True
equilibration_simulation.steps_per_iteration = 100000
equilibration_simulation.total_number_of_iterations = 1
equilibration_simulation.output_frequency = 5000
equilibration_simulation.checkpoint_frequency = 10
equilibration_simulation.timestep = 2.0 * unit.femtosecond
equilibration_simulation.thermodynamic_state = ProtocolPath("thermodynamic_state", "global")
equilibration_simulation.thermostat_friction = 1.0 / unit.picosecond
equilibration_simulation.input_coordinate_file = ProtocolPath("output_coordinate_file", energy_minimization.id)
equilibration_simulation.parameterized_system = ProtocolPath("parameterized_system", assign_parameters.id)
equilibration_simulation.enable_pbc = True
equilibration_simulation.allow_gpu_platforms = True
equilibration_simulation.high_precision = False

# Step 6: Analysis step to check if density has converged
density_analysis = AverageObservable("density_analysis")
density_analysis.observable = ProtocolPath("observables.Density", equilibration_simulation.id)
density_analysis.thermodynamic_state = ProtocolPath("thermodynamic_state", "global")
density_analysis.input_coordinate_file = ProtocolPath("output_coordinate_file", equilibration_simulation.id)
density_analysis.input_trajectory_path = ProtocolPath("trajectory_file_path", equilibration_simulation.id)
density_analysis.output_observable = "density"

# Step 7: Define a conditional group for final production simulation
final_production = OpenMMSimulation("final_production_simulation")
final_production.input_coordinate_file = ProtocolPath("output_coordinate_file", equilibration_simulation.id)
final_production.input_force_field = ProtocolPath("parameterized_system", assign_parameters.id)
final_production.steps = 50000  # 0.1 ns run for testing
final_production.thermodynamic_state = ProtocolPath("thermodynamic_state", "global")
final_production.parameterized_system = ProtocolPath("parameterized_system", assign_parameters.id)

# Add a unique identifier to the conditional group and its protocols
conditional_group = ConditionalGroup("conditional_group")
conditional_group.add_protocols(final_production, density_analysis)

# Set up a condition which will check if the density uncertainty is less than some threshold.
condition = ConditionalGroup.Condition()
condition.condition_type = ConditionalGroup.Condition.Type.LessThan
condition.right_hand_value = 0.5 * unit.grams / unit.milliliter
condition.left_hand_value = ProtocolPath("value.error", density_analysis.id)

# Add the condition.
conditional_group.add_condition(condition)

# Step 8: Define schema and metadata
schema = WorkflowSchema()
schema.protocol_schemas = [
    build_coordinates.schema,
    assign_parameters.schema,
    energy_minimization.schema,
    equilibration_simulation.schema,
    conditional_group.schema
]

metadata = {
    "substance": substance,
    "thermodynamic_state": ThermodynamicState(
        temperature=298.15 * unit.kelvin,
        pressure=1.0 * unit.atmosphere
    ),
    "force_field_path": "openff_unconstrained-2.1.0.offxml"
}

# Step 9: Create the workflow from schema
workflow = Workflow.from_schema(schema, metadata=metadata)

# # Step 10: Execute the workflow
# workflow.execute()

ValueError: The conditional_group protocol tries to take input from a non-existent protocol: density_analysis.value.error

In [100]:

# Step 1: Convert SMILES to a molecule and add hydrogens
benzene_smiles = "c1ccccc1"
benzene_mol = Chem.AddHs(Chem.MolFromSmiles(benzene_smiles))
rdDistGeom.EmbedMolecule(benzene_mol)

# Convert molecule to SMILES again (to ensure it's correctly hydrogenated)
smiles = Chem.MolToSmiles(benzene_mol)
substance = Substance()
substance.add_component(Component(smiles=smiles), MoleFraction(1.0))

# Step 2: Build coordinates
build_coordinates = BuildCoordinatesPackmol("build_coordinates")
build_coordinates.allow_merging = True
build_coordinates.count_exact_amount = True
build_coordinates.mass_density = 0.95 * unit.grams / unit.milliliters
build_coordinates.box_aspect_ratio = [1.0, 1.0, 1.0]
build_coordinates.substance = ProtocolPath("substance", "global")
build_coordinates.tolerance = 2.0 * unit.angstrom
build_coordinates.verbose_packmol = False
build_coordinates.retain_packmol_files = False

# Step 3: Assign parameters
assign_parameters = BuildSmirnoffSystem("assign_parameters")
assign_parameters.allow_merging = True
assign_parameters.force_field_path = ProtocolPath("force_field_path", "global")
assign_parameters.coordinate_file_path = ProtocolPath("coordinate_file_path", build_coordinates.id)
assign_parameters.substance = ProtocolPath("substance", "global")

# Step 4: Energy minimization
energy_minimization = OpenMMEnergyMinimisation("energy_minimisation")
energy_minimization.allow_merging = True
energy_minimization.input_coordinate_file = ProtocolPath("coordinate_file_path", build_coordinates.id)
energy_minimization.parameterized_system = ProtocolPath("parameterized_system", assign_parameters.id)
energy_minimization.tolerance = 10.0 * unit.kilojoules_per_mole / unit.nanometer
energy_minimization.enable_pbc = True
energy_minimization.max_iterations = 0

# Step 5: Equilibration simulation
equilibration_simulation = OpenMMSimulation("equilibration_simulation")
equilibration_simulation.allow_merging = True
equilibration_simulation.steps_per_iteration = 100000
equilibration_simulation.total_number_of_iterations = 1
equilibration_simulation.output_frequency = 5000
equilibration_simulation.checkpoint_frequency = 10
equilibration_simulation.timestep = 2.0 * unit.femtosecond
equilibration_simulation.thermodynamic_state = ProtocolPath("thermodynamic_state", "global")
equilibration_simulation.thermostat_friction = 1.0 / unit.picosecond
equilibration_simulation.input_coordinate_file = ProtocolPath("output_coordinate_file", energy_minimization.id)
equilibration_simulation.parameterized_system = ProtocolPath("parameterized_system", assign_parameters.id)
equilibration_simulation.enable_pbc = True
equilibration_simulation.allow_gpu_platforms = True
equilibration_simulation.high_precision = False


# Step 6: Analysis step to check if density fluctuation has converged
density_analysis = AverageObservable("density_analysis")
density_analysis.observable = ProtocolPath("observables.Density", equilibration_simulation.id)
density_analysis.thermodynamic_state = ProtocolPath("thermodynamic_state", "global")
density_analysis.input_coordinate_file = ProtocolPath("output_coordinate_file", equilibration_simulation.id)
density_analysis.input_trajectory_path = ProtocolPath("trajectory_file_path", equilibration_simulation.id)
density_analysis.output_observable = "density_fluctuation"

# Step 7: Define a conditional group for iterative equilibration simulation and analysis
conditional_group = ConditionalGroup("conditional_group")
conditional_group.add_protocols(equilibration_simulation, density_analysis)

# Set up a condition which will check if the density fluctuation is less than some threshold.
condition = ConditionalGroup.Condition()
condition.condition_type = ConditionalGroup.Condition.Type.LessThan
condition.right_hand_value = 0.1 * unit.grams / unit.milliliter  # Threshold for fluctuation
condition.left_hand_value = ProtocolPath("value.error", density_analysis.id)

# Add the condition.
conditional_group.add_condition(condition)
# Step 8: Final production simulation after density fluctuation has converged
final_production = OpenMMSimulation("final_production_simulation")
final_production.input_coordinate_file = ProtocolPath("output_coordinate_file", equilibration_simulation.id)
final_production.input_force_field = ProtocolPath("parameterized_system", assign_parameters.id)
final_production.steps = 50000  # 0.1 ns run for testing
final_production.thermodynamic_state = ProtocolPath("thermodynamic_state", "global")
final_production.parameterized_system = ProtocolPath("parameterized_system", assign_parameters.id)

# Step 9: Define schema and metadata
schema = WorkflowSchema()
schema.protocol_schemas = [
    build_coordinates.schema,
    assign_parameters.schema,
    energy_minimization.schema,
    conditional_group.schema,
    final_production.schema
]

metadata = {
    "substance": substance,
    "thermodynamic_state": ThermodynamicState(
        temperature=298.15 * unit.kelvin,
        pressure=1.0 * unit.atmosphere
    ),
    "force_field_path": "openff_unconstrained-2.1.0.offxml"
}

# Step 10: Create the workflow from schema
workflow = Workflow.from_schema(schema, metadata=metadata)

# Step 11: Execute the workflow
# workflow.execute()


ValueError: The conditional_group protocol tries to take input from a non-existent protocol: density_analysis.value.error